In [2]:
import pathlib
from ensembl_lite._download import download_ensembl_tree
from ensembl_lite._species import Species
from cogent3 import load_tree

In [2]:
def find_common_name_and_unknown_species(tree):
    ## looking up common name from latin name
    tree_name = pathlib.Path(tree).name
    download_kwargs = kwargs.copy()
    download_kwargs["tree_fname"] = tree_name

    tree = download_ensembl_tree(**download_kwargs)
    tip_names = tree.get_tip_names()

    # Assume tree and Species are already defined and imported correctly
    tip_names = tree.get_tip_names()
    common_names = {}
    unknown_species = {}
    # Loop through all tip names to get common names
    for name in tip_names:
        try:
            common_name = Species.get_common_name(name)
            common_names[name] = common_name
        except ValueError:
            unknown_species[name] = 'Unknown'
            common_names[name] = 'Unknown'
    return common_names, unknown_species


In [3]:
paths = list(pathlib.Path("/Users/gulugulu/repos/PuningAnalysis/data/dataset2_ensemble_trees/raw_data/tree_nh_file").glob("**/*.nh"))


In [4]:
path_mammal = '/Users/gulugulu/repos/PuningAnalysis/data/dataset2_ensemble_trees/raw_data/tree_nh_file/91_eutherian_mammals_EPO-Extended_default.nh'
tree_mammal= load_tree(path_mammal, format=None, underscore_unmunge=False)
tree_mammal.get_figure(show_support=True, threshold=0.8, height = 2500, width = 1500)

In [51]:
path = '/Users/gulugulu/repos/PuningAnalysis/data/dataset2_ensemble_trees/raw_data/tree_nh_file/vertebrates_species-tree_Ensembl.nh'
tree_vertebrates= load_tree(path, format=None, underscore_unmunge=False)
outgroup_species = ['aquila_chrysaetos_chrysaetos']
from ensembl_lite._species import Species

def make_db_prefixes():
    return [n.lower().replace(" ", "_") for n in Species.get_species_names()]

db_prefixes = make_db_prefixes()

def find_db_prefix(name):
    for db_prefix in db_prefixes:
        if name.startswith(db_prefix):
            return db_prefix
    return None

import json
with open ('/Users/gulugulu/repos/PuningAnalysis/results/output_data/available_species_name.json', 'r') as infile:
    available_species_names = json.load(infile)

with open ('/Users/gulugulu/repos/PuningAnalysis/results/output_data/common_names_mapping.json',  'r') as common_name_infile:
    common_names_mapping = json.load(common_name_infile)

db_names = []
probs = []
for tip_name in tree_vertebrates.get_tip_names():
    db_name = find_db_prefix(tip_name.lower())
    if db_name is None:
        probs.append(tip_name)
    else:
        db_names.append(db_name)

db_name_dict = {}
for tip_name in tree_vertebrates.get_tip_names():
    db_name = find_db_prefix(tip_name.lower())
    if db_name is None:
        db_name_dict[tip_name] = tip_name
    else:
        db_name_dict[tip_name] = db_name

for key in db_name_dict.keys():
    if key.split('_')[0:2] == ['Mus', 'musculus']:
        if key != 'Mus_musculus_reference_CL57BL6_strain':
            db_name_dict[key] = 'mus_musculus_strain'
        else:
            db_name_dict[key] = 'mus_musculus'

for key in db_name_dict.keys():
    if key.split('_')[0:2] == ['Sus','scrofa']:
        if key != 'Sus_scrofa_reference_breed':
            db_name_dict[key] = 'sus_scrofa_strain'
        else:
            db_name_dict[key] = 'sus_scrofa'

tree_vertebrates.reassign_names(db_name_dict)
tree_vertebrates.reassign_names(common_names_mapping)

In [69]:
seq_path = '/Users/gulugulu/repos/PuningAnalysis/data/ensembl_ortholog_sequences/homologies_sequence_common_name/ENSG00000162620.json'


In [3]:
from cogent3.util.deserialise import deserialise_object
from clock_project.genome_analysis.traids_selection import extract_species_names
from clock_project.genome_analysis.homolog_analysis import codon_aligner, cpos3, loader, filter
from cogent3 import get_app
import json

In [174]:
sequences = deserialise_object(json.load(open(seq_path, 'r')))
species_names = sequences.names
tip_names = species_names + outgroup_species
tree_seq = tree_vertebrates.get_sub_tree(tip_names)
tree_topology = tree_seq.get_sub_tree(species_names)

alignment = codon_aligner(sequences.trim_stop_codons(strict=True))
just3rd_aligned = cpos3(alignment)
just3rd_aligned_no_degenerates = just3rd_aligned.no_degenerates(motif_length=3)



NameError: name 'seq_path' is not defined

In [307]:

model_fit_result_path = '/Users/gulugulu/repos/PuningAnalysis/results/output_data/model_fitting_result/ENSG00000162543.json'
load_json_app = get_app("load_json")
result_lf = load_json_app(model_fit_result_path)
ens_tree = result_lf.get_ens_tree()
ens_tree.get_figure(show_support=True, threshold=0.8, height = 1500, width = 1000)

In [196]:
import numpy as np
def get_ingroup_pair(alignment):
    species_names = alignment.names
    ingroup_species_pair = np.random.choice(np.array(species_names), 2, replace=False)

    return list(ingroup_species_pair)

In [197]:
def get_relationship(ingroup_species_pair, ens_tree):
    lca_node = ens_tree.lowest_common_ancestor(ingroup_species_pair).get_node_names()[0]
    parents = ens_tree.lowest_common_ancestor(ingroup_species_pair).parent
    if parents != None:
        parent_node = parents.get_node_names()[0]
        parent_children = ens_tree.get_nodes_dict()[parent_node].children
        relation_dict = {'sibling': [], 'cousion': []}
        for child in parent_children:
            grad_child = child.get_node_names()
            if grad_child[0] == lca_node:
                relation_dict['sibling'].extend(child.get_tip_names())
            else:
                relation_dict['cousion'].extend(child.get_tip_names())
        return relation_dict
    else:
        return None


In [198]:
alignment_path = '/Users/gulugulu/repos/PuningAnalysis/data/ensembl_ortholog_sequences/homologies_alignments_common_name/ENSG00000162620.json'
alignment = deserialise_object(json.load(open(alignment_path, 'r')))

In [199]:
triads_species_names = {'ingroup1': 'Vervet_AGM', 'ingroup2': 'Cat', 'outgroup': 'Armadillo'}
tree_topology_triads = ens_tree.get_sub_tree(triads_species_names.values())
model_triads = get_app("model", "GN", tree = tree_topology_triads, time_het="max", optimise_motif_probs=True)

In [200]:
sequence_path = '/Users/gulugulu/repos/PuningAnalysis/data/ensembl_ortholog_sequences/homologies_sequence_common_name/ENSG00000162620.json'

sequences = deserialise_object(json.load(open(sequence_path, 'r')))


In [245]:
result_lf_dir = '/Users/gulugulu/repos/PuningAnalysis/results/output_data/model_fitting_result'

In [255]:
import os
import glob
path1 = '/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_alignment_new/ENSG00000160050/'
file_name = os.path.basename(path1).rsplit('/', 1)[0]
print(file_name)
triads_alignment_paths = glob.glob(os.path.join(path1, "*.json"))
result_lf_path = os.path.join(result_lf_dir, f"{file_name}.json")
result_lf = load_json_app(result_lf_path)

In [260]:
file_name = os.path.basename(path1.rstrip('/'))
file_name

'ENSG00000160050'

In [243]:
triads_alignment_dir = '/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_alignment_new'
paths_aln = glob.glob(os.path.join(triads_alignment_dir, '*/'))

In [240]:
file_name

'ENSG00000004455'

In [201]:
traid_sequences = sequences.take_seqs(triads_species_names.values())

In [229]:
from clock_project.genome_analysis.homolog_analysis import cpos3, codon_aligner


def get_just3rd_aligned_no_degenerates(seqs):
    """Processes a sequence file to align and filter third codon positions without degenerates."""
    seqs_no_stop_codon = seqs.trim_stop_codons(strict=True)
    aligned = codon_aligner(seqs_no_stop_codon)
    just3rd_aligned = cpos3(aligned)
    just3rd_aligned_no_degenerates = just3rd_aligned.no_degenerates(motif_length=3)
    return just3rd_aligned_no_degenerates

In [236]:
triads_alignment_1 = get_just3rd_aligned_no_degenerates(traid_sequences)

In [268]:
triads_alignment_1

,0
Armadillo,GTTAACAAGATCTAAGTCTTACTAATAACGTTAGCTCTTAATGATGAATTTTAGCCCATT
Vervet_AGM,..........CT..........A.....TT..G.......G.....GTCC...A....C.
Cat,........A.CT........C.G...G.TT..G..............C.C.G.A....C.


In [237]:
triads_alignment_1.names

['Vervet_AGM', 'Cat', 'Armadillo']

In [204]:
triads_alignment_2 = alignment.take_seqs(triads_species_names.values())

In [205]:
result1 = model_triads(triads_alignment_1)
result2 = model_triads(triads_alignment_2)

In [271]:
result1.lf.tree.lowest_common_ancestor([triads_species_names['ingroup1'], triads_species_names['ingroup2']]).get_node_names()[0]

'Boreoeutheria'

In [207]:
result2.lf

GN
log-likelihood = -398.9384
number of free parameters = 51
========================================================================
edge             parent           length       A>C        A>G        A>T
------------------------------------------------------------------------
Armadillo        root             0.1182    0.0000    15.8352     6.0752
Cat              Boreoeutheria    0.0842    1.4756     5.2257     0.0000
Vervet_AGM       Boreoeutheria    0.0409    0.0000    50.0000    18.3847
Boreoeutheria    root             0.0285    0.0000     0.0000    50.0000
------------------------------------------------------------------------

continued: 
======================================================================
   C>A       C>G        C>T        G>A       G>C        G>T        T>A
----------------------------------------------------------------------
0.0000    0.0000    32.8032    27.9591    0.0000     0.0000     0.0000
2.2924    0.0000     5.7170     3.1411    0.0000     0.0000     0.0000
0.0000    0.0000     0.0000     0.0000    0.0000     0.0000    12.7550
0.0000    0.0000    50.0000    50.0000    0.0000    39.2973    19.5761
----------------------------------------------------------------------

continued: 
=======
    T>C
-------
 8.2252
 3.9065
20.5118
 0.0000
-------

====================================
     A         C         G         T
------------------------------------
0.3214    0.1710    0.1139    0.3937
------------------------------------

In [208]:
nodes = ens_tree.get_nodes_dict()
sub_tree = ens_tree.get_sub_tree(triads_species_names.values())
sub_tree.get_figure()


In [225]:
get_relationship(ingroup_species_pair, ens_tree)

{'sibling': ['American_black_bear',
  'Giant_panda',
  'Ferret',
  'Dog',
  'Red_fox',
  'Cat',
  'Lion',
  'Donkey',
  'Vaquita',
  'Dolphin',
  'Blue_whale',
  'Goat',
  'Domestic_yak',
  'American_bison',
  'Siberian_musk_deer',
  'Yarkand_deer',
  'Pig',
  'Chacoan_peccary',
  'Arabian_camel',
  'Alpaca',
  'Greater_horseshoe_bat',
  'Kangaroo_rat',
  'Ryukyu_mouse',
  'Rat',
  'Golden_Hamster',
  'Prairie_vole',
  'Northern_American_deer_mouse',
  'Long_tailed_chinchilla',
  'Degu',
  'Guinea_Pig',
  'Squirrel',
  'Alpine_marmot',
  'Eurasian_red_squirrel',
  'Rabbit',
  'Chimpanzee',
  'Gorilla',
  'Sumatran_orangutan',
  'Gibbon',
  'Golden_snub_nosed_monkey',
  'Sooty_mangabey',
  'Drill',
  'Olive_baboon',
  'Macaque',
  'Vervet_AGM',
  'Panamanian_white_faced_capuchin',
  'Bolivian_squirrel_monkey',
  "Ma's_night_monkey",
  'White_tufted_ear_marmoset',
  'Tarsier',
  'Bushbaby',
  'Tree_Shrew'],
 'cousion': ['Armadillo']}

In [209]:
sub_tree

Tree("(Armadillo,(Cat,'Vervet_AGM')Boreoeutheria)root;")

In [211]:
ens_dict = {n: sub_tree.to_rich_dict()['edge_attributes'][n]['length'] for n in triads_species_names.values()}

In [212]:
ens_dict

{'Vervet_AGM': 0.04412841189885737,
 'Cat': 0.07816712914152998,
 'Armadillo': 0.1420254581405398}

In [214]:
ingroup_species_pair = [triads_species_names['ingroup1'], triads_species_names['ingroup2']]

In [215]:
parent_node = ens_tree.lowest_common_ancestor(ingroup_species_pair).parent.get_node_names()[0]
lca_node = ens_tree.lowest_common_ancestor(ingroup_species_pair).get_node_names()[0]

In [216]:
rate_matrices_triads2 = {n: result_lf.get_rate_matrix_for_edge(n, calibrated=True) for n in triads_species_names.values()}

In [217]:
def get_trace_back_nodes(ingroup_species_gene_name, outgroup, lca_node, parent_node, ens_tree):
    trace_back_nodes = {}
    for species in ingroup_species_gene_name:
        path = []
        for tree in ens_tree.get_connecting_edges(lca_node, species):
            internal_root = tree.get_node_names()[0]
            path.append(internal_root)
        trace_back_nodes[species] = path

    outgroup_path = []
    for tree in ens_tree.get_connecting_edges(parent_node, outgroup):
        internal_root = tree.get_node_names()[0]
        outgroup_path.append(internal_root)
    trace_back_nodes[outgroup] = outgroup_path

    return trace_back_nodes

In [218]:
def get_Q_matrices(triads_species_names, lca_node, parent_node, ens_tree, result_lf, ens_dict):
    ingroup_species = [triads_species_names['ingroup1'], triads_species_names['ingroup2']]
    outgroup = triads_species_names['outgroup']
    trace_back_nodes = get_trace_back_nodes(ingroup_species, outgroup, lca_node, parent_node, ens_tree)
    branch_length_dict = {species: {n: ens_tree.to_rich_dict()['edge_attributes'][n]['length'] for n in trace_back_nodes[species]} for species in triads_species_names.values()}    
    matrices_dict = {species: {n: result_lf.get_rate_matrix_for_edge(n, calibrated=True).to_array() for n in trace_back_nodes[species]} for species in triads_species_names.values()}
    p_matrices = {}
    for species in triads_species_names.values(): 
        p_matrices[species] = {}
        for node_name, matrix in matrices_dict[species].items():
            p_matrices[species][node_name] = scipy.linalg.expm(matrix * branch_length_dict[species][node_name])
    result_p_matrices = {}
    for species in triads_species_names.values(): 
        result_p_matrix = list(p_matrices[species].values())[0]
        for matrix in list(p_matrices[species].values())[1:]:
            result_p_matrix = np.dot(result_p_matrix, matrix)
        result_p_matrices[species] = result_p_matrix
    result_Qt_matrices = {species: scipy.linalg.logm(matrix) for species, matrix in result_p_matrices.items()}
    result_Q_matrices = {species: result_Qt_matrices[species]/ens_dict[species] for species in triads_species_names.values()}

    return result_Q_matrices

In [219]:
ingroup_species = [triads_species_names['ingroup1'], triads_species_names['ingroup2']]
outgroup = triads_species_names['outgroup']
trace_back_nodes = get_trace_back_nodes(ingroup_species, outgroup, lca_node, parent_node, ens_tree)

In [220]:
trace_back_nodes

{'Vervet_AGM': ['Boreoeutheria',
  'Euarchontoglires.2',
  'Euarchontoglires',
  'Primates',
  'Haplorrhini',
  'Simiiformes',
  'Catarrhini',
  'Cercopithecidae',
  'Cercopithecinae.5',
  'Vervet_AGM'],
 'Cat': ['Boreoeutheria',
  'Laurasiatheria.3',
  'Laurasiatheria.2',
  'Laurasiatheria',
  'Carnivora',
  'Felidae',
  'Cat'],
 'Armadillo': ['Eutheria.2', 'Armadillo']}

In [232]:
from clock_project.simulation.wts import calculate_non_stationarity
from cogent3.maths.measure import jsd



def get_triads_info(triads_species_names, ens_tree, sequence, result_lf, lca_node, parent_node):
    sub_tree = ens_tree.get_sub_tree(triads_species_names.values())
    ingroup_species_gene_name = [triads_species_names['ingroup1'], triads_species_names['ingroup2']]
    outgroup = triads_species_names['outgroup']
    ingroup_seqs = sequence.take_seqs(ingroup_species_gene_name)
    nuc_freqs = ingroup_seqs.probs_per_seq()
    ingroup_jsd = jsd(nuc_freqs[ingroup_species_gene_name[0]], nuc_freqs[ingroup_species_gene_name[1]])
    distribution_internal_root = list(result_lf.get_motif_probs_by_node()[lca_node])
    distribution_root = list(result_lf.get_motif_probs_by_node()[parent_node])
    ens_dict = {n: sub_tree.to_rich_dict()['edge_attributes'][n]['length'] for n in triads_species_names.values()}
    ens_difference = np.sqrt((ens_dict[ingroup_species_gene_name[0]]-ens_dict[ingroup_species_gene_name[1]])**2)
    shortest_ens = min(ens_dict.values())  
    triad_sequence = sequence.take_seqs(triads_species_names.values())
    triad_alignment = get_just3rd_aligned_no_degenerates(triad_sequence)
    matrices = get_Q_matrices(triads_species_names, lca_node, parent_node, ens_tree, result_lf, ens_dict)
    nabla_dict = {}
    for species in ingroup_species_gene_name:
        nabla_dict[species] = calculate_non_stationarity(distribution_internal_root, matrices[species], ens_dict[species])
    
    nabla_dict[outgroup] = calculate_non_stationarity(distribution_root, matrices[outgroup], ens_dict[outgroup])
    triad_data = {'triads_species_names': triads_species_names, 'triads_info_big_tree': 
        {'internal_node': lca_node,
         'root': parent_node,
        'ingroup_jsd': ingroup_jsd,
        'ens_difference': ens_difference,
        'shortest_ingroup_ens': shortest_ens,
        'ens': ens_dict,
        'internal_root_distribution': distribution_internal_root,
        'root_distribution': distribution_root,
        'nabla_values': nabla_dict}}
    
    return triad_data, triad_alignment

In [233]:
matrices = get_Q_matrices(triads_species_names, lca_node, parent_node, ens_tree, result_lf, ens_dict)



In [234]:
traid_info = get_triads_info(
   triads_species_names, ens_tree, sequences, result_lf, lca_node, parent_node
)

traid_info[0]

{'triads_species_names': {'ingroup1': 'Vervet_AGM',
  'ingroup2': 'Cat',
  'outgroup': 'Armadillo'},
 'triads_info_big_tree': {'internal_node': 'Boreoeutheria',
  'root': 'Eutheria.2',
  'ingroup_jsd': 0.0035286444750179946,
  'ens_difference': 0.034038717242672614,
  'shortest_ingroup_ens': 0.04412841189885737,
  'ens': {'Vervet_AGM': 0.04412841189885737,
   'Cat': 0.07816712914152998,
   'Armadillo': 0.1420254581405398},
  'internal_root_distribution': [0.4039741240439267,
   0.17448172346622864,
   0.3108684352076752,
   0.11067571728216971],
  'root_distribution': [0.4155761298210775,
   0.17448172329851894,
   0.299493919991724,
   0.11044822688867967],
  'nabla_values': {'Vervet_AGM': 0.5501733102511572,
   'Cat': 0.39590857339503965,
   'Armadillo': 0.25066639863458745}}}

In [272]:
triads_info_dir = '/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_new'
triads_info_path = os.path.join(triads_info_dir, file_name, "triads_species_names_dict.json")

In [273]:
triads_info_path

'/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_new/ENSG00000160050/triads_species_names_dict.json'

In [304]:
{n: result1.lf.get_rate_matrix_for_edge(n, calibrated=True) for n in triads_species_names.values()}

{'Vervet_AGM': =============================================
            T          C          A          G
 ---------------------------------------------
 T    -0.8518     0.5054     0.3204     0.0260
 C     0.0000    -0.0000     0.0000     0.0000
 A     0.4313     0.0000    -1.7296     1.2984
 G     0.9796     0.0000     0.0000    -0.9796
 ---------------------------------------------,
 'Cat': =============================================
            T          C          A          G
 ---------------------------------------------
 T    -0.8723     0.6952     0.0000     0.1771
 C     0.9709    -1.3526     0.3817     0.0000
 A     0.0000     0.2547    -1.1655     0.9108
 G     0.0000     0.0000     0.4592    -0.4593
 ---------------------------------------------,
 'Armadillo': =============================================
            T          C          A          G
 ---------------------------------------------
 T    -0.4501     0.3645     0.0000     0.0856
 C     1.6562    -1.8967

In [305]:
11.574221161219732/(11.574221161219732+1.000000028068235e-06+1.0000000015597518e-06)

0.9999998272022234

In [303]:
{n: result1.lf.get_rate_matrix_for_edge(n, calibrated=False) for n in triads_species_names.values()}

{'Vervet_AGM': =============================================
            T          C          A          G
 ---------------------------------------------
 T    -0.0372     0.0221     0.0140     0.0011
 C     0.0000    -0.0000     0.0000     0.0000
 A     0.0189     0.0000    -0.0756     0.0568
 G     0.0428     0.0000     0.0000    -0.0428
 ---------------------------------------------,
 'Cat': =============================================
            T          C          A          G
 ---------------------------------------------
 T    -0.0689     0.0549     0.0000     0.0140
 C     0.0767    -0.1069     0.0302     0.0000
 A     0.0000     0.0201    -0.0921     0.0720
 G     0.0000     0.0000     0.0363    -0.0363
 ---------------------------------------------,
 'Armadillo': =============================================
            T          C          A          G
 ---------------------------------------------
 T    -0.0609     0.0493     0.0000     0.0116
 C     0.2242    -0.2567

In [296]:
matrix = result1.lf.get_rate_matrix_for_edge('Vervet_AGM', calibrated=True).to_array().tolist()

armadillo
              T                       C                         A                    G
T             -              4.258415989819681       1.0000007612834815e-06          -  
C   19.347510658093984                -              1.0000000440791968e-06  2.8093771796703635
A   5.947541747155204        1.000000326392643e-06              -            7.530325514557298
G   1.000000028068235e-06    1.0000000015597518e-06  11.574221161219732              - 

In [314]:
{n: result1.lf.get_psub_for_edge(n) for n in triads_species_names.values()}

{'Vervet_AGM': =========================================
           T         C         A         G
 -----------------------------------------
 T    0.9636    0.0217    0.0132    0.0015
 C    0.0000    1.0000    0.0000    0.0000
 A    0.0190    0.0002    0.9273    0.0535
 G    0.0412    0.0005    0.0003    0.9581
 -----------------------------------------,
 'Cat': =========================================
           T         C         A         G
 -----------------------------------------
 T    0.9353    0.0504    0.0010    0.0133
 C    0.0703    0.9008    0.0274    0.0015
 A    0.0007    0.0182    0.9135    0.0675
 G    0.0000    0.0003    0.0341    0.9656
 -----------------------------------------,
 'Armadillo': =========================================
           T         C         A         G
 -----------------------------------------
 T    0.9458    0.0422    0.0007    0.0112
 C    0.1920    0.7781    0.0019    0.0280
 A    0.0621    0.0015    0.8605    0.0760
 G    0.0041    0.

In [317]:
triads_alignment_1

,0
Armadillo,GTTAACAAGATCTAAGTCTTACTAATAACGTTAGCTCTTAATGATGAATTTTAGCCCATT
Vervet_AGM,..........CT..........A.....TT..G.......G.....GTCC...A....C.
Cat,........A.CT........C.G...G.TT..G..............C.C.G.A....C.


In [331]:
result1.lf.get_motif_probs_by_node()

root,0.3762,0.1767,0.3221,0.1250
Armadillo,0.4102,0.1539,0.2923,0.1436
Boreoeutheria,0.3922,0.1710,0.3156,0.1212
Cat,0.3791,0.1796,0.2975,0.1438
Vervet_AGM,0.3889,0.1796,0.2979,0.1336


In [349]:
result1.lf.get_param_rules()

[{'par_name': 'G>T',
  'edge': 'Armadillo',
  'init': 1.000000028068235e-06,
  'lower': 1e-06,
  'upper': 50},
 {'par_name': 'G>T',
  'edge': 'Boreoeutheria',
  'init': 1.00170197598609e-06,
  'lower': 1e-06,
  'upper': 50},
 {'par_name': 'G>T',
  'edge': 'Cat',
  'init': 1.0000000047241149e-06,
  'lower': 1e-06,
  'upper': 50},
 {'par_name': 'G>T',
  'edge': 'Vervet_AGM',
  'init': 37.72459126925222,
  'lower': 1e-06,
  'upper': 50},
 {'par_name': 'G>C',
  'edge': 'Armadillo',
  'init': 1.0000000015597518e-06,
  'lower': 1e-06,
  'upper': 50},
 {'par_name': 'G>C',
  'edge': 'Boreoeutheria',
  'init': 1.000025446597216e-06,
  'lower': 1e-06,
  'upper': 50},
 {'par_name': 'G>C',
  'edge': 'Cat',
  'init': 1.0000001189210458e-06,
  'lower': 1e-06,
  'upper': 50},
 {'par_name': 'G>C',
  'edge': 'Vervet_AGM',
  'init': 1.00001259161911e-06,
  'lower': 1e-06,
  'upper': 50},
 {'par_name': 'G>A',
  'edge': 'Armadillo',
  'init': 11.574221161219732,
  'lower': 1e-06,
  'upper': 50},
 {'par_na

In [350]:
result1.lf.get_statistics()

[edge params
 edge             parent           length       A>C        A>G        A>T
 ------------------------------------------------------------------------
 Armadillo        root             0.1354    0.0000     7.5303     5.9475
 Cat              Boreoeutheria    0.0790    1.4385     5.1431     0.0000
 Vervet_AGM       Boreoeutheria    0.0437    0.0000    50.0000    16.6078
 Boreoeutheria    root             0.0208    0.0000     0.0000    50.0000
 ------------------------------------------------------------------------
 
 continued: edge params
    C>A       C>G        C>T        G>A       G>C        G>T        T>A
 ----------------------------------------------------------------------
 0.0000    2.8094    19.3475    11.5742    0.0000     0.0000     0.0000
 2.1556    0.0000     5.4825     2.5933    0.0000     0.0000     0.0000
 0.0000    0.0000     0.0000     0.0000    0.0000    37.7246    12.3392
 0.0000    0.0000    50.0000    50.0000    0.0000     0.0000     0.0000
 ----------

In [353]:
result1.lf.get_all_rate_matrices(calibrated=True)

{('Armadillo',): =============================================
            T          C          A          G
 ---------------------------------------------
 T    -0.4501     0.3645     0.0000     0.0856
 C     1.6562    -1.8967     0.0000     0.2405
 A     0.5091     0.0000    -1.1538     0.6446
 G     0.0000     0.0000     0.9908    -0.9908
 ---------------------------------------------,
 ('Boreoeutheria',): =============================================
            T          C          A          G
 ---------------------------------------------
 T    -0.0317     0.0000     0.0000     0.0317
 C     1.5839    -1.5839     0.0000     0.0000
 A     1.5839     0.0000    -1.5839     0.0000
 G     0.0000     0.0000     1.5839    -1.5839
 ---------------------------------------------,
 ('Cat',): =============================================
            T          C          A          G
 ---------------------------------------------
 T    -0.8723     0.6952     0.0000     0.1771
 C     0.970